In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as geom_nn
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset

In [12]:
dataset = TUDataset(root='../../data/datasets', name='MUTAG')

In [13]:
class GNNClassifier(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GNNClassifier, self).__init__()
        self.conv1 = geom_nn.GraphConv(num_features, 16)
        self.conv2 = geom_nn.GraphConv(16, 32)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = geom_nn.global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNClassifier(dataset.num_features, dataset.num_classes).to(device)


In [15]:
train_dataset = dataset[:len(dataset) // 2]
test_dataset = dataset[len(dataset) // 2:]
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

/home/stan/Documents/projects/ADS/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()

def test():
    model.eval()
    correct = 0
    total = 0
    for data in test_loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        _, predicted = torch.max(output.data, 1)
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()
    accuracy = 100 * correct / total
    return accuracy


In [17]:
num_epochs = 10
for epoch in range(num_epochs):
    train()
    accuracy = test()
    print(f'Epoch {epoch + 1}, Accuracy: {accuracy:.2f}%')

Epoch 1, Accuracy: 60.64%
Epoch 2, Accuracy: 60.64%
Epoch 3, Accuracy: 60.64%
Epoch 4, Accuracy: 60.64%
Epoch 5, Accuracy: 60.64%
Epoch 6, Accuracy: 60.64%
Epoch 7, Accuracy: 60.64%
Epoch 8, Accuracy: 60.64%
Epoch 9, Accuracy: 60.64%
Epoch 10, Accuracy: 60.64%
